In [1]:
#pip install pandahouse  # если библиотека ещё не установлена
import pandahouse as ph
# Здесь презентация: CR ARPU students AB test
import time # для иззмерения времени запроса

In [2]:
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [3]:
# создаем запрос, db автоматически заменится на значение из database
# нет особого смысла в AND toMonth(timest) = 10, но в условии прописано про "в этом месяце"
# и вообще там тогда должно быть AND toMonth(timest) = toMonth(now()), но на апрель 2023 года в БД записей нет - выводит 0
# сделал 2 запроса
start_time = time.time() # начало исполнения запроса

q = '''
--SELECT MAX(timest) FROM default.peas -- 2021-10-31 17:29:52.000 самая поздняя дата в таблице
--SELECT MIN(timest) FROM default.peas -- 2021-10-30 13:00:00.000 самая ранняя дата в таблице

SELECT
    COUNT(diligence) AS count_of_eager_students
FROM(
SELECT 
    SUM(correct) AS diligence
FROM default.peas
GROUP BY st_id
HAVING diligence >= 20 AND toMonth(timest) = 10) AS t
    '''

# отправляем запрос и записываем результат в пандасовский датафрейм
diligence = ph.read_clickhouse(query=q, connection=connection_default)

end_time = time.time() # конец исполнения запроса
elapsed_time = end_time - start_time #  время запроса

print("Время выполнения запроса: {:.2f} секунд".format(elapsed_time))

diligence

Время выполнения запроса: 0.20 секунд


,count_of_eager_students
0,136


In [4]:
# создаем запрос, db автоматически заменится на значение из database
# нет особого смысла в AND toMonth(timest) = 10, но в условии прописано про "в этом месяце"
# и вообще там тогда должно быть AND toMonth(timest) = toMonth(now()), но на апрель 2023 года в БД записей нет - выводит 0
start_time = time.time() # начало исполнения запроса
q = '''
--SELECT MAX(timest) FROM default.peas -- 2021-10-31 17:29:52.000 самая поздняя дата в таблице
--SELECT MIN(timest) FROM default.peas -- 2021-10-30 13:00:00.000 самая ранняя дата в таблице

SELECT COUNT(st_id) AS count_of_eager_students
FROM (
    SELECT st_id
    FROM (
        SELECT st_id, SUM(correct) AS diligence
        FROM default.peas
        WHERE toMonth(timest) = 10
        GROUP BY st_id
    ) AS t
    WHERE diligence >= 20
) AS t2
    '''

# отправляем запрос и записываем результат в пандасовский датафрейм
diligence = ph.read_clickhouse(query=q, connection=connection_default)

end_time = time.time() # конец исполнения запроса
elapsed_time = end_time - start_time #  время запроса

print("Время выполнения запроса: {:.2f} секунд".format(elapsed_time))

diligence

Время выполнения запроса: 0.06 секунд


,count_of_eager_students
0,136


In [5]:
# создаем запрос, db автоматически заменится на значение из database
q = '''
-- табличное выражение для расчета метрик
WITH cte1 as (
SELECT
    studs.st_id as id,
    studs.test_grp as experiment_group,
    check.money as payment,
    check.subject as discipline,
    peas_count.points_a as points_a,
    peas_count.math_points_a as math_points_a
FROM default.studs as studs
LEFT JOIN default.final_project_check as check
    ON studs.st_id = check.st_id
LEFT JOIN ( 
        SELECT -- таблица с айди студентов и количеством баллов (горошин) по всем предметам и только по математике
            st_id,
            SUM(correct) as points_a,
            SUMIf(correct, subject = 'Math') as math_points_a
        FROM default.peas
        GROUP BY st_id)  as peas_count
    ON studs.st_id = peas_count.st_id
) 

-- расчет метрик
SELECT 
    experiment_group as Experiment_group,
    ROUND(SUM(payment) / uniqExact(id), 2) as ARPU, -- средний чек на 1 пользователя
    ROUND(SUMIf(payment, points_a > 10 )/uniqIf(id, points_a > 10), 2) as ARPU_a, -- средний чек на 1 активного пользователя
    ROUND((uniqIf(id, payment > 0) / uniq(id))*100, 2) as CR, -- общая конверсия в покупку
    ROUND((uniqIf(id, payment > 0 AND points_a > 10) / uniqIf(id, points_a > 10))*100, 2) as CR_a, -- конверсия активных пользователей в покупку
    ROUND((uniqIf(id, payment > 0 AND math_points_a >= 2 AND discipline = 'Math') / uniqIf(id, math_points_a >= 2))*100, 2) as CR_math_a --конверсия пользователей, активных по математике  в покупку курса по математике 
FROM cte1
GROUP BY experiment_group
    '''

# отправляем запрос и записываем результат в пандасовский датафрейм
metrics = ph.read_clickhouse(query=q, connection=connection_default)
metrics

,Experiment_group,ARPU,ARPU_a,CR,CR_a,CR_math_a
0,control,4540.98,10393.70,4.92,11.02,6.12
1,pilot,11508.47,29739.58,10.85,26.04,9.52
